# A hypothetical case study for stochastic optimization: the classic farmer problem

A farmer wants to plant three types of crops: Wheat, Corn, and Beans. He has a fixed amount of land and aims to meet the demand for each type of crop in the local market. The demand for each crop is uncertain. The yield of each crop is uncertain and depends on the amount of water they receive during the season (endogenous uncertainty). Additionally, the market prices for each crop at the end of the season are uncertain (exogenous uncertainty).

The farmer can choose to invest in an irrigation system (with three possible operating levels: Low, Medium, High) that influences the amount of water each crop receives. The investment and operational costs vary with the level.

### Nomenclature:

### Set:

- $i \in \in \{1, 2, 3\}$: Category of produce, Wheat, Corn, and Beans respectively.

- $j \in \{1, 2, 3\}$: Irrigation operating level(low, medium, and high respectively).

#### Decision Variables:

- $x_{i,j}$: Acres of land planted with produce i under irrigation operating level j.

- $y_{i,j}$ ($j \in \{1, 2, 3\}$): Binary, decision for produce i under irrigation operating level j.

#### Intermediate variables:

- $z_i$: Wheat, Corn and Beans product

#### Uncertain Parameters:

- $\xi_{i, j}$: Yields for produce i with irrigation operating level j(endogenous uncertainty).
- $P_i$: Market prices for Wheat, Corn, and Beans at the end of the season (exogenous uncertainty).
- $D_i$: Market demand for Wheat, Corn, and Beans (exogenous uncertainty).

#### Probability distribution of uncertain parameters(when realized in scenario s):

- $\Psi^{s}$

#### Deterministic parameters

- $R_{i}$ unit cost of seeds for Wheat, Corn, and Beans, respectively.

- $C^{f}_{i}$ fixed cost of irrigation operating level i.

- $C^{v}_{i, }$ variable cost of irrigation operating level i.

- $L$: total land available.

In [2]:
from pyomo.environ import *

### Deterministic Formulation

#### Constraints:

1. **Total Land Constraint**:
\begin{align*}
\sum_{i,j} x_{i,j} \le L
\end{align*}

2. **Demand Constraint**:
\begin{align*}
z_i &\le D_i \\
\end{align*}

3. **Irrigation Choice Constraints I**:
\begin{align*}
\sum_{j} y_{i,j} = 1 \quad \forall i
\end{align*}

4. **Irrigation Choice Constraints II**:
\begin{align*}
x_{i,j} \le Ly_{i,j} \quad \forall i,j
\end{align*}

5. **Yield Constraints**:

\begin{align*}
\sum_{j} \xi_{i,j} x_{i,j} = z_i
\end{align*}

#### Objective:

$$ \textbf{min} \quad \sum_{i}  \{R_{i}\sum_{j} {x_{i,j}} + \sum_{j} C^{f}_{i,j} {y_{i,j}} + \sum_{j} C^{v}_{i,j} {x_{i,j}} \} - \sum_{i} D_{i} {z_{i}}$$

### Stochastic formulation given scenarios S

#### Constraints:

1. **Total Land Constraint(1st stage)**:
\begin{align*}
\sum_{i,j} x_{i,j} \le L
\end{align*}

2. **Demand Constraint(2nd stage)**
\begin{align*}
z_{i,s} &\le D_{i,s} \quad \forall s\\
\end{align*}

3. **Irrigation Choice Constraints I(1st stage)**:
\begin{align*}
\sum_{j} y_{i,j} = 1 \quad \forall i
\end{align*}

4. **Irrigation Choice Constraints II(1st stage)**:
\begin{align*}
x_{i,j} \le Ly_{i,j} \quad \forall i,j
\end{align*}

5. **Yield Constraints(2nd stage)**:

\begin{align*}
\sum_{j} \xi_{i,j,s} x_{i,j} = z_{i,s} \quad  \forall  i,s
\end{align*}

#### Objective:

$$ \textbf{min} \quad \sum_{i}  \{R_{i}\sum_{j} {x_{i,j}} + \sum_{j} C^{f}_{i,j} {y_{i,j}} + \sum_{j} C^{v}_{i,j} {x_{i,j}} \} - \sum_{s} \Psi_{s} \sum_{i} D_{i,s} {z_{i,s}}$$

In [7]:
# Instantiate the model
m2 = ConcreteModel()

# Sets
m2.i = Set(initialize=[1,2,3])  # Produce category
m2.j = Set(initialize=[1,2,3])  # Irrigation level
m2.s = Set(initialize=[1,2])  # This needs to be initialized with your scenarios

# Decision Variables
m2.x = Var(m2.i, m2.j, domain=NonNegativeReals)  # Land allocated
m2.y = Var(m2.i, m2.j, domain=Binary)            # Irrigation choice
m2.z = Var(m2.i, m2.s, domain=NonNegativeReals)  # Production based on uncertainty realization

# Parameters

# Cost of seeds
m2.R = Param(m2.i, initialize={1:100, 2:80, 3:90})

# Fixed cost of irrigation
m2.Cf = Param(m2.i, m2.j, initialize={
    (1, 1): 10, (1, 2): 20, (1, 3): 30,
    (2, 1): 8, (2, 2): 16, (2, 3): 24,
    (3, 1): 12, (3, 2): 22, (3, 3): 32
})

# Variable cost of irrigation
m2.Cv = Param(m2.i, m2.j, initialize={
    (1, 1): 10, (1, 2): 12, (1, 3): 15,
    (2, 1): 8,  (2, 2): 9,  (2, 3): 11,
    (3, 1): 11, (3, 2): 13, (3, 3): 14,
})

# Sample Yields for each produce type under each irrigation level and scenario
m2.xi = Param(m2.i, m2.j, m2.s, initialize={
    (1, 1, 1): 50, (1, 2, 1): 55, (1, 3, 1): 58,
    (2, 1, 1): 60, (2, 2, 1): 62, (2, 3, 1): 65,
    (3, 1, 1): 45, (3, 2, 1): 47, (3, 3, 1): 50,
    (1, 1, 2): 52, (1, 2, 2): 56, (1, 3, 2): 59,
    (2, 1, 2): 61, (2, 2, 2): 63, (2, 3, 2): 67,
    (3, 1, 2): 46, (3, 2, 2): 48, (3, 3, 2): 52,
})

# Market prices for each produce and scenario
m2.P = Param(m2.i, m2.s, initialize={
    (1, 1): 200, (1, 2): 210,
    (2, 1): 170, (2, 2): 175,
    (3, 1): 220, (3, 2): 230,
})

# Market demand for each produce and scenario
m2.D = Param(m2.i, m2.s, initialize={
    (1, 1): 5000, (1, 2): 5500,
    (2, 1): 6000, (2, 2): 6500,
    (3, 1): 4500, (3, 2): 4800,
})

# Total land available
m2.L = Param(initialize=10000)

# Probability of scenarios:

m2.Psi = Param(m2.s, initialize={1: 0.5, 2: 0.5})

# Constraints
# Total Land Constraint
def land_constraint_rule(model):
    return sum(model.x[i,j] for i in model.i for j in model.j) <= model.L
m2.land_constraint = Constraint(rule=land_constraint_rule)

# Irrigation Choice Constraint I
def irrigation_choice_I_rule(model, i):
    return sum(model.y[i,j] for j in model.j) == 1
m2.irrigation_choice_I = Constraint(m2.i, rule=irrigation_choice_I_rule)

# Irrigation Choice Constraint II
def irrigation_choice_II_rule(model, i, j):
    return model.x[i,j] <= model.L * model.y[i,j]
m2.irrigation_choice_II = Constraint(m2.i, m2.j, rule=irrigation_choice_II_rule)

# Demand Constraint
def demand_constraint_rule(model, i, s):
    return model.z[i,s] <= model.D[i,s]
m2.demand_constraint = Constraint(m2.i, m2.s, rule=demand_constraint_rule)

# Yield Constraint
def yield_constraint_rule(model, i, s):
    return sum(model.xi[i,j,s]*model.x[i,j] for j in model.j) == model.z[i,s]
m2.yield_constraint = Constraint(m2.i, m2.s, rule=yield_constraint_rule)

# Objective Function
def obj_rule(model):
    return sum(model.R[i]*sum(model.x[i,j] for j in model.j) + sum(model.Cf[i,j]*m2.y[i,j] + model.Cv[i,j]*model.x[i,j] for j in model.j) for i in model.i) \
           - sum(model.Psi[s]*sum(model.P[i,s]*model.z[i,s] for i in model.i) for s in model.s)
m2.obj = Objective(rule=obj_rule, sense=minimize)

opt2 = SolverFactory('gurobi_persistent')
opt2.set_instance(m2)
opt2.solve()